# Оценка необходимого размера выборки, эффекта 
К сожалению, наши коллеги из отдела машинного обучения провели АB-тест, но забыли посоветоваться с нами заранее. В результате время проведения АB-теста было выбрано по наитию, одна неделя. </br>
Не самое плохое решение, неделя, как минимум, полностью покрывает недельную сезонность в данных. Но хотелось бы для дальнейших экспериментов оценить, какой размер выборки нам нужен заранее. 

In [1]:
# импортируем библиотеки
import pandas as pd
import pandahouse
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.stats.power import TTestIndPower
from random import randint
from db.read_db import get_df
import math
import statistics

# достаем данные из БД для дальнейшего анализа
## открываем файл с запросом
with open('./db/ab_test.sql') as f:
    query = f.read()
## читаем данные из БД при помощи функции get_df()
df = get_df(query)

# сформируем данные для дальнейшего анализа
group_a = df.loc[df['exp_group']==0, 'ctr'].values
group_b = df.loc[df['exp_group']==1, 'ctr'].values

1. При помощи библиотеки метода TTestIndPower оценить, какой размер выборки нам потребуется в дальнейшем если мы:
     - Хотим использовать t-test для сравнения CTR по юзерам
     - Ожидаем, что размер эффекта равен 0.3
     - Хотим, чтобы мощность нашего теста равнялась 0.8.

In [2]:
# Оценка размера выборки
analysis = TTestIndPower()

effect = 0.3
alpha = 0.05
power = 0.8
# Результат – наблюдения на одну выборку (соотвественно, умножаем на 2)
size = analysis.solve_power(effect_size=effect, power=power, alpha=alpha, ratio=1, nobs1=None)
size_rounded = math.ceil(size)

message = 'Если мы хотим использовать t-test для сравнения CTR по юзерам, '
message = message + '\nесли мы ожидаем размер эффекта, равный {}'.format(effect)
message = message + '\nи хотим иметь мощность теста, равную {},'.format(power)
message = message + '\nминимальный размер выборки должен составлять {}'.format(size_rounded)
message = message + '\nв каждой из групп'
print('1. Оценка размера выборки')
print('--- --- --- --- --- ---')
print(message)
print('=== === === === === === === === === === === ===')

1. Оценка размера выборки
--- --- --- --- --- ---
Если мы хотим использовать t-test для сравнения CTR по юзерам, 
если мы ожидаем размер эффекта, равный 0.3
и хотим иметь мощность теста, равную 0.8,
минимальный размер выборки должен составлять 176
в каждой из групп
=== === === === === === === === === === === ===


2. Рассчитайте, чему оказался равен размер эффекта в нашем проведенном АB-тесте, и сделайте вывод, могли бы использовать меньший размер выборки при сохранении мощности на уровне 0.8.

Для расчета эффекта используем подход, изложенный в [Когда останавливать A/B-тест? Часть 1: MDE](https://medium.com/statistics-experiments/%D0%BA%D0%BE%D0%B3%D0%B4%D0%B0-%D0%BE%D1%81%D1%82%D0%B0%D0%BD%D0%B0%D0%B2%D0%BB%D0%B8%D0%B2%D0%B0%D1%82%D1%8C-a-b-%D1%82%D0%B5%D1%81%D1%82-%D1%87%D0%B0%D1%81%D1%82%D1%8C-1-mde-7d39b668b488) c применением pooled SD:</br>
- SDpooled = √((n1-1).SD12 + (n2-1).SD22)/(n1+n2-2) where,</br>
    - SD1 = Standard Deviation for group 1</br>
    - SD2 = Standard Deviation for group 2</br>
    - n1 =  Sample Size for group 1</br>
    - n2 =  Sample Size for group 2</br>

In [3]:
def get_sd_pooled(sample1, sample2):
    '''
    Расчет pooled standard deviation
    ---
    Параметры:
        sample1  -- 1я группа АВ теста
        sample2  -- 2я группа АВ теста
    '''
    SD1 = statistics.stdev(sample1)
    SD2 = statistics.stdev(sample2)   
    n1 = len(sample1)
    n2 = len(sample2)
    
    sd_pooled = math.sqrt(
                      (
                          (n1 - 1) * SD1 * SD1 + 
                          (n2-1) * SD2 * SD2
                      ) / (n1 + n2 - 2))
    
    return sd_pooled

def get_effect_size(sample1, sample2):
    '''
    Расчет размер эффекта (Effect Size) в нашем проведенном АB-тесте.
    Порядок контроля или теста не важен.
    ---
    Параметры:
        sample1  -- 1я группа АВ теста
        sample2  -- 2я группа АВ теста
    '''
    sd_pooled = get_sd_pooled(sample1, sample2)
    mean1 = sample1.mean()
    mean2 = sample2.mean()
    max_mean = max(mean1, mean2)
    min_mean = min(mean1, mean2)
    mde = (max_mean - min_mean) / sd_pooled
    return mde

# расчитываем эффект от АВ теста
ab_effect = get_effect_size(group_a, group_b)

# подсчет ratio
ab_ratio = len(group_b) / len(group_a)

# расчет размера выборки после уточнения эффекта
size_upd = analysis.solve_power(
    effect_size=ab_effect, 
    power=power, 
    alpha=alpha, 
    ratio=ab_ratio, 
    nobs1=None)
size_upd_rounded_1 = math.ceil(size_upd)
size_upd_rounded_2 = math.ceil(size_upd_rounded_1 * ab_ratio)

print('2.1. Размер эффекта в нашем проведенном АB-тесте:')
print('--- --- --- --- --- ---')
message = 'Размер эффекта равен {:.2f}'.format(get_effect_size(group_a, group_b))
print(message)

print('\n2.2. Размер выборки при сохранении мощности на уровне 0.8:')
print('--- --- --- --- --- ---')
message = 'Если мы хотим использовать t-test для сравнения CTR по юзерам, '
message = message + '\nесли мы ожидаем размер эффекта, равный {:.2f}'.format(ab_effect)
message = message + '\nи хотим иметь мощность теста, равную {},'.format(power)
message = message + '\nминимальный размер выборки в 1й группе должен составлять {}'.format(size_upd_rounded_1)
message = message + '\nи во 2й группе {}'.format(size_upd_rounded_2)
print(message)
message = 'У нас сейчас выборки составили {} и {} наблюдений.'.format(len(group_a), 
                                                                     len(group_b))
print(message)

2.1. Размер эффекта в нашем проведенном АB-тесте:
--- --- --- --- --- ---
Размер эффекта равен 0.16

2.2. Размер выборки при сохранении мощности на уровне 0.8:
--- --- --- --- --- ---
Если мы хотим использовать t-test для сравнения CTR по юзерам, 
если мы ожидаем размер эффекта, равный 0.16
и хотим иметь мощность теста, равную 0.8,
минимальный размер выборки в 1й группе должен составлять 590
и во 2й группе 591
У нас сейчас выборки составили 12997 и 13002 наблюдений.


Мы могли бы использовать меньший размер выборки для АВ теста.